<a href="https://colab.research.google.com/github/taeyoung02/taeyoung02.github.io/blob/master/imdb%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import imdb
import numpy as np

In [0]:
(train_data, train_labels), (test_data, test_labels)=imdb.load_data(num_words=10000)

In [0]:
max([max(sequence) for sequence in train_data])

In [0]:
train_data,len(train_data)

In [0]:
train_labels

In [0]:
test_data, len(test_data)

In [0]:
#원래 문장으로 바꿔보기
word_index=imdb.get_word_index()
word_index

In [0]:
reverse_word_index = dict(
    [(value,key) for (key, value) in word_index.items()]
)

In [0]:
decoded_review=' '.join(
    [reverse_word_index.get(i-3, '?') for i in train_data[0]]
)
decoded_review

In [0]:
#[1,3,5,6]=[0,1,0,1,0,1,1]
#신경망에 숫자리스트를 넣을수없기 때문에 텐서를 넣어주는데
#원핫인코딩으로 바꿔준다
def vectorize_sequences(sequences, dimension=10000):
    results=np.zeros((len(sequences), dimension))#크기가 (len(sequences), dimension)이고 모든원소가 0
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1. #원핫인코딩(result[i][j]만 1이고 나머지 다 0)
    return results

x_train=vectorize_sequences(train_data)
x_test=vectorize_sequences(test_data)

In [0]:
x_train,len(x_train)

In [0]:
#벡터로 변환
y_train=np.asarray(train_labels).astype('float32')
y_test=np.asarray(test_labels).astype('float32')
y_train,y_test

In [0]:
from tensorflow.python.keras import models
from tensorflow.python.keras import layers

In [0]:
model=models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))#은닉층
#model.add(layers.Dense(16, activation='relu'))#은닉층 은닉유닛 16개 = 16차원. 신경망의 데이터의 복잡도
model.add(layers.Dense(1, activation='sigmoid'))#출력층

In [0]:
model.compile(optimizer='rmsprop',#rmsprop 경사하강법(빠름)
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
x_val=x_train[:10000]
partial_x_train=x_train[10000:]
y_val=y_train[:10000]
partial_y_train=y_train[10000:]

In [0]:
history=model.fit(partial_x_train,
                  partial_y_train,
                  epochs=20,
                  batch_size=512,
                  validation_data=(x_val, y_val))

In [0]:
import matplotlib.pyplot as plt

In [0]:
history_dict=history.history
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs=range(1, len(loss) +1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
loss

In [0]:
plt.clf()
acc=history_dict['accuracy']
val_acc = history_dict['val_accuracy']

plt.plot(epochs, acc ,'bo', label='Traning acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

epoch을 20이나 해버려서 훈련데이터가 검증데이터에 적용이안되고    
훈련데이터에 과대적합되버림   
epoch을 낮춰본다 

In [0]:
model=models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=4, batch_size=512)

In [0]:
results=model.evaluate(x_test,y_test)
results 

손실과 정확도 반환

In [0]:
y_hat=model.predict(x_test)
y_hat

In [0]:
y_test

**정리**   
이진분류문제에서 네트워크는 하나의 유닛과(차원) sigmoid함수를 가진   
Dense층으로 끝나야 한다. (0과1사이의 스칼라 값이기 때문)   
이진분류문제의 손실함수는 binary_crossentropy   
